In [7]:
import pandas as pd
from streaming_optimizer import optimize_streaming_packages
from datetime import datetime

In [42]:
games_raw = pd.read_csv('data/bc_game.csv') # some games and when they are played
streaming_packages_raw = pd.read_csv('data/bc_streaming_package.csv') # streaming packages with price
streaming_offers_raw = pd.read_csv('data/bc_streaming_offer.csv')  # which service offers which game, live? on-demand-highlights?
games_raw

,id,team_home,team_away,starts_at,tournament_name
0,1,Deutschland,Schottland,2024-06-14 19:00:00,Europameisterschaft 2024
1,2,Ungarn,Schweiz,2024-06-15 13:00:00,Europameisterschaft 2024
2,3,Spanien,Kroatien,2024-06-15 16:00:00,Europameisterschaft 2024
3,4,Italien,Albanien,2024-06-15 19:00:00,Europameisterschaft 2024
4,5,Polen,Niederlande,2024-06-16 13:00:00,Europameisterschaft 2024
...,...,...,...,...,...
8866,8872,San Tirso SD,Selaya FC,2024-10-09 19:00:00,Copa del Rey 24/25
8867,8873,UD San Pedro,CD Ceuta 6 de Junio,2024-10-09 19:00:00,Copa del Rey 24/25
8868,8874,CD Sonseca,Cp Parla Escuela,2024-10-09 19:00:00,Copa del Rey 24/25
8869,8875,CD Gevora,UD Playas de Sotavento,2024-10-09 19:00:00,Copa del Rey 24/25


In [ ]:
# Extract unique package IDs
packages = streaming_packages_raw['id'].unique().tolist()

# Extract unique game IDs
# games = games_raw['id'].unique().tolist() # should be passed to FE

# Create game_dates dictionary
games_raw['starts_at'] = pd.to_datetime(games_raw['starts_at'])
game_dates = games_raw.set_index('id')['starts_at'].to_dict()


# Create C_month and C_year dictionaries, dropping packages with NA for the respective type
C_month = streaming_packages_raw.dropna(subset=['monthly_price_cents']) \
    .set_index('id')['monthly_price_cents'].to_dict()
C_year = streaming_packages_raw.dropna(subset=['monthly_price_yearly_subscription_in_cents']) \
    .set_index('id')['monthly_price_yearly_subscription_in_cents'].to_dict()

# Create P_g dictionary
P_g = streaming_offers_raw.groupby('game_id')['streaming_package_id'].apply(list).to_dict()

# optimize_streaming_packages(packages, [1,2,3],game_dates, C_month, C_year, P_g)

PulpSolverError: Pulp: Error while executing C:\Users\tmari\AppData\Roaming\Python\Python312\site-packages\pulp\solverdir\cbc\win\64\cbc.exe

In [4]:
from streaming_optimizer import optimize_streaming_packages
from datetime import datetime, timedelta

# Example Input Data
packages = ["P1", "P2"]
games = ["G1", "G2", "G3", "G4", "G5", 'G6', 'G7']
game_dates = {
    "G1": datetime(2023, 1, 15),
    "G2": datetime(2023, 2, 20),
    "G3": datetime(2023, 3, 25),
    "G4": datetime(2023, 4, 10),
    "G5": datetime(2023, 4, 20),
    "G6": datetime(2024, 1, 10),
    "G7": datetime(2024, 3, 15),
}
C_month = {'P1': 20}
C_year = {'P1': 180}
P_g = {'G1': ['P1'], 'G2': ['P1'], 'G3': ['P1', 'P2'], 'G4': ['P1', 'P2'], 'G5': ['P1', 'P2'],
    'G6': ['P1'], 'G7': ['P1', 'P2']}
optimize_streaming_packages(packages, games, game_dates, C_month, C_year, P_g)



Status: Optimal
Total Cost: 100.0
Active Monthly Subscriptions:
  Package: P1, Start Date: 2023-01-15
  Package: P1, Start Date: 2023-02-20
  Package: P1, Start Date: 2023-03-25
  Package: P1, Start Date: 2024-01-10
  Package: P1, Start Date: 2024-03-15
Active Yearly Subscriptions:
